In [1]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

## 1. Preparation
### 1.1 Prepare LLM

In [2]:
from llama_index.llms import Ollama

OLLAMA_HOST = 'localhost'
OLLAMA_MODEL = 'mistral'
llm = Ollama(model=OLLAMA_MODEL, base_url="http://"+OLLAMA_HOST+":11434")

In [3]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm, 
    embed_model="local",
    # chunk_size=512
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 Prepare Graph Store
`Neo4j` is supported as a graph store integration. You can persist, visualize, and query graphs using LlamaIndex and Neo4j. Furthermore, existing Neo4j graphs are directly supported using `text2cypher` and the `KnowledgeGraphQueryEngine`.

If you’ve never used Neo4j before, you can download the desktop client [here](https://neo4j.com/download/).

Once you open the client, create a new project and install the `apoc` integration. Full instructions here. Just click on your project, select `Plugins` on the left side menu, install APOC and restart your server.

In [38]:
import os

In [13]:
%pip install neo4j

username = "neo4j"
password = os.environ["NEO4J_PASSWORD"]
# Neo4j cloud has a generous free tier, so I use that instead of localhost
# url = "bolt://localhost:7687"
url = "bolt+s://3b2530f1.databases.neo4j.io:7687"
database = "neo4j"

You should consider upgrading via the '/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from llama_index.graph_stores import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

In [15]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 2. Build the Knowledge Graph
### 2.1 Preprocess Data

In [16]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

### 2.2 Extract Triplets and Save to Graph

In [17]:
from llama_index import KnowledgeGraphIndex

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=2,
    include_embeddings=True,
)

 (Guardians of the Galaxy Vol. 3, is a 2023 American superhero film)
(Guardians of the Galaxy Vol. 3, produced by Marvel Studios)
(Guardians of the Galaxy Vol. 3, distributed by Walt Disney Studios Motion Pictures)
(Guardians of the Galaxy Vol. 3, based on the Marvel Comics superhero team Guardians of the Galaxy)
(Guardians of the Galaxy Vol. 3, sequel to Guardians of the Galaxy and Guardians of the Galaxy Vol. 2)
(Guardians of the Galaxy Vol. 3, written and directed by James Gunn)
(Guardians of the Galaxy Vol. 3, features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone)
(Guardians of the Galaxy Vol. 3, Guardians must save Rocket's life from the High Evolutionary)
(James Gunn, announced his return to write and direct in April 2017)
(Disney fired him from the film in July 2018)
(Disney reversed course by that Oct

Open https://workspace-preview.neo4j.io/workspace/query

## 3. Create VectorStoreIndex for RAG


In [18]:
from llama_index.vector_stores import Neo4jVectorStore

embed_dim = 1536

neo4j_vector = Neo4jVectorStore(username, password, url, embed_dim)

In [21]:
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

storage_context_vector = StorageContext.from_defaults(vector_store=neo4j_vector)
vector_index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context_vector, 
    service_context=service_context,
)

## 4. (Optional) Persist and Load from disk Llama Indexes
### 4.1. Persist

### 4.2. Restore
In Llama Index, there are two scenarios we could apply Graph RAG:

- Build Knowledge Graph from documents with Llama Index, with LLM or even local models, to do this, we should go for `KnowledgeGraphIndex`.

- Leveraging existing Knowledge Graph, in this case, we should use `KnowledgeGraphRAGQueryEngine`.



In [36]:
index_name = "existing_index"
text_node_property = "text"
existing_vector = Neo4jVectorStore(
    username,
    password,
    url,
    embed_dim,
    index_name=index_name,
    text_node_property=text_node_property,
)
vector_index = VectorStoreIndex.from_vector_store(
    existing_vector,
    service_context=service_context,
)

## 5. Prepare for different query approaches

### 5.1 Graph RAG query engine

In [37]:
# in case we just built the index and we have it available
kg_rag_query_engine = index.as_query_engine(
    include_text=False, 
    retriever_mode="keyword",
    response_mode="tree_summarize"
)

In [25]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

kg_rag_query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, 
    service_context=service_context,
    r
)

### 5.2. Vector RAG query engine

In [32]:
vector_rag_query_engine = vector_index.as_query_engine()

### 5.3 Graph+Vector RAG query engine

In [28]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [29]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

In [30]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

## 6. Query with the engines

In [26]:
from IPython.display import Markdown, display

In [27]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

Entities processed: ['Quill', 'Peter Quill', 'Tell me', 'referring', 'Peter Quill (specifically if referring to a particular character from Marvel Comics or Movies)', 'specifically', 'particular', 'About', 'Movies', 'Marvel', 'character', 'Tell', 'Comics', 'Peter']
Entities processed: ['Marvel Comics', 'Quill', 'Peter Quill', 'Peter Quill (Marvel)', 'Tell', 'specifically', 'About', 'Movies', 'Marvel', 'particularly', 'character', 'Referring.', 'Comics', 'Peter', 'Marvel character']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
['KEEPING', "James Gunn's script"]
['SPEAKS_THE_UNCENSORED_WORD_"FUCK"_IN', 'Vol. 3']
['SPEAKS_THE_UNCENSORED_WORD_"FUCK"_IN', 'Vol. 3', 'FILMED_AT_THE_SAME_TIME_AS', 'Guardians of the Galaxy Holiday Special']


<b> Based on the provided context information, Peter Quill is not directly mentioned as a subject. However, we know that "James Gunn's script" for "Vol. 3" and "Guardians of the Galaxy Holiday Special" were filmed at the same time as Volume 3, which contains the predicate "FILMED_AT_THE_SAME_TIME_AS". Since Peter Quill is a character in the Guardians of the Galaxy franchise, it is reasonable to infer that he is likely present in these productions. Therefore, we can answer the query with:

Peter Quill is a character in the Guardians of the Galaxy franchise, and he appeared in the production of "Vol. 3" and the "Guardians of the Galaxy Holiday Special", which were filmed at the same time.</b>

In [33]:
response_vector_rag = vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_vector_rag}</b>"))

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.queryNodes`: Caused by: java.lang.IllegalArgumentException: Index query vector has 384 dimensions, but indexed vectors have 1536.}

In [31]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

INFO:llama_index.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.


<b> Peter Quill, also known as Star-Lord, is the leader of the Guardians of the Galaxy. He was abducted from Earth as a child and raised by a group of alien thieves and smugglers called the Ravagers. In the film "Guardians of the Galaxy Vol. 3," Quill is in a state of depression following the appearance of an alternate version of his deceased lover Gamora, who does not share the same affection for him as her older version did, affecting his leadership of the Guardians. Chris Pratt portrays Peter Quill in the film.</b>

## 7. Comparison and Conclusion

In [32]:
analysis = llm.complete(f"""
Compare the QA results on "Tell me about Peter Quill.", list the knowledge facts between them, to help evalute them. Output in markdown table.

Result from Graph: {response_graph_rag}
---
Result from Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

""")

In [33]:
display(Markdown(analysis.text))

 | Fact | Graph Result | Vector Result | Both Results |
| --- | --- | --- | --- |
| Character Name | Peter Quill / Star-Lord | Peter Quill / Star-Lord | Peter Quill / Star-Lord |
| Universe | Marvel Comics | Marvel Cinematic Universe | Marvel Universe |
| Creation | Created by Dan Abnett and Andy Lanning, first appeared in "Annihilators" #1 in July 2006 | Portrayed by Chris Pratt in films since 2014 | Created, first appearance in respective universes |
| Species | Human from Earth | Human who was abducted from Earth as a child and grew up among extraterrestrial beings | Human, abducted from Earth as a child, raised among aliens |
| Birth Year | Born in 1982 | N/A | Born in 1982 |
| Backstory | Raised among the Ravagers, skilled mercenary and thief, leads Guardians of the Galaxy | Skilled in combat and piloting spaceships, quirky sense of humor, love for classic Earth music, depicted as more traditional superhero compared to the Guardians | Raised among the Ravagers, became a space adventurer, joined the Guardians of the Galaxy |
| Weaponry | Star-Lord gun, Orb, Milano spaceship | N/A | Advanced weaponry, including Star-Lord gun and Orb, Milano spaceship |
| Personality | Charming, rebellious, sarcastic, strong sense of loyalty to those he cares about, romantically involved with Gamora and Rocket Raccoon | Quirky sense of humor, love for classic Earth music, depicted as a more traditional superhero compared to the Guardians | Charming, rebellious, sarcastic, romantic relationships with Gamora and Rocket Raccoon |
| Films | N/A | Appeared in "Guardians of the Galaxy" (2014), "Guardians of the Galaxy Vol. 2" (2017), "Avengers: Infinity War" (2018), and "Avengers: Endgame" (2018) | N/A, appears in films |
| Director | N/A | James Gunn was fired from the franchise over old tweets, but negotiations to return ultimately failed | N/A, no mention of director |
| Depiction in "Vol. 3" | In a state of depression following Gamora's appearance | N/A | In a state of depression following Gamora's appearance in "Guardians of the Galaxy Vol. 3" |